In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

## 2014 Ebola Outbreak Data

[Caitlin Rivers' `ebola` GitHub repository](https://github.com/cmrivers/ebola) contains summarized reports of Ebola cases from three countries during the recent outbreak of the disease in West Africa. These data are licenced for both commercial and non-commercial use. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

![ebola](images/ebola.jpg)

From these data files, we will use pandas to import them and create a single data frame that includes the **daily totals of new cases** for each country. 

In [2]:
url_base = 'https://raw.githubusercontent.com/cmrivers/ebola/master/guinea_data/'

In [4]:
pd.read_csv(url_base+'2014-09-02.csv').head(3)

,Date,Description,Totals,Conakry,Gueckedou,Macenta,Dabola,Kissidougou,Dinguiraye,Telimele,...,Mzerekore,Yomou,Dubreka,Forecariah,Kerouane,Coyah,Dalaba,Beyla,Kindia,Lola
0,2014-09-02,New cases of suspects,11,NaN,5,6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-09-02,New cases of probables,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-09-02,New cases of confirmed,14,NaN,4,6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Clearly, we will need to develop row **masks** to extract the data we need across all files, without having to manually extract data from each file.

Let's hack at one file to develop the mask.

In [5]:
sample = pd.read_csv(url_base+'2014-09-02.csv')

To prevent issues with capitalization, we will simply revert all labels to lower case.

In [6]:
lower_vars = sample.Description.str.lower()
lower_vars

0                               new cases of suspects
1                              new cases of probables
2                              new cases of confirmed
3                   total new cases registered so far
4                             total cases of suspects
5                            total cases of probables
6                            total cases of confirmed
7        cumulative (confirmed + probable + suspects)
8                     total suspected non-class cases
9                               new deaths registered
10                           total deaths of suspects
11                          total deaths of probables
12                          total deaths of confirmed
13    total deaths (confirmed + probables + suspects)
14          fatality rate for confirmed and probables
15        new cases of confirmed among health workers
16       total case of confirmed among health workers
17         new deaths registered among health workers
18       total deaths regist

Since we are interested in extracting new cases only, we can use the **string accessor** attribute to look for key words that we would like to include or exclude.

In [7]:
case_mask = (lower_vars.str.contains('new') 
             & (lower_vars.str.contains('case') | lower_vars.str.contains('suspect')) 
             & ~lower_vars.str.contains('non')
             & ~lower_vars.str.contains('total'))

We could have instead used regular expressions to do the same thing.

Finally, we are only interested in three columns.

In [9]:
sample.loc[case_mask, ['Date', 'Description', 'Totals']]

,Date,Description,Totals
0,2014-09-02,New cases of suspects,11
1,2014-09-02,New cases of probables,0
2,2014-09-02,New cases of confirmed,14
15,2014-09-02,New cases of confirmed among health workers,0


We can now embed this operation in a loop over all the filenames in the database. We first need to create a range of dates, since the data files are organized by date.

In [10]:
outbreak_dates = [str(date).split(' ')[0] for date in pd.date_range('2014-07-01', '2014-11-30')]
outbreak_dates

['2014-07-01',
 '2014-07-02',
 '2014-07-03',
 '2014-07-04',
 '2014-07-05',
 '2014-07-06',
 '2014-07-07',
 '2014-07-08',
 '2014-07-09',
 '2014-07-10',
 '2014-07-11',
 '2014-07-12',
 '2014-07-13',
 '2014-07-14',
 '2014-07-15',
 '2014-07-16',
 '2014-07-17',
 '2014-07-18',
 '2014-07-19',
 '2014-07-20',
 '2014-07-21',
 '2014-07-22',
 '2014-07-23',
 '2014-07-24',
 '2014-07-25',
 '2014-07-26',
 '2014-07-27',
 '2014-07-28',
 '2014-07-29',
 '2014-07-30',
 '2014-07-31',
 '2014-08-01',
 '2014-08-02',
 '2014-08-03',
 '2014-08-04',
 '2014-08-05',
 '2014-08-06',
 '2014-08-07',
 '2014-08-08',
 '2014-08-09',
 '2014-08-10',
 '2014-08-11',
 '2014-08-12',
 '2014-08-13',
 '2014-08-14',
 '2014-08-15',
 '2014-08-16',
 '2014-08-17',
 '2014-08-18',
 '2014-08-19',
 '2014-08-20',
 '2014-08-21',
 '2014-08-22',
 '2014-08-23',
 '2014-08-24',
 '2014-08-25',
 '2014-08-26',
 '2014-08-27',
 '2014-08-28',
 '2014-08-29',
 '2014-08-30',
 '2014-08-31',
 '2014-09-01',
 '2014-09-02',
 '2014-09-03',
 '2014-09-04',
 '2014-09-

In [11]:
datasets = []
for country in ('guinea', 'sl', 'liberia'):
    
    print('Getting {} data files'.format(country))
    for date in outbreak_dates:
        
        file = 'https://raw.githubusercontent.com/cmrivers/ebola/master/{}_data/{}.csv'.format(country, date)
        try:
            data = pd.read_csv(file)
            print('\t{}'.format(date))
        except Exception:
            continue
        
        # Convert to lower case to avoid capitalization issues
        data.columns = data.columns.str.lower()
        # Column naming is inconsistent. These procedures deal with that.
        keep_columns = ['date']
        if 'description' in data.columns:
            keep_columns.append('description')
        else:
            keep_columns.append('variable')
            
        if 'totals' in data.columns:
            keep_columns.append('totals')
        else:
            keep_columns.append('national')
            
        # Index out the columns we need, and rename them
        keep_data = data[keep_columns]
        keep_data.columns = 'date', 'variable', 'totals'
        
        # Extract the rows we might want
        lower_vars = keep_data.variable.str.lower()
        # Of course we can also use regex to do this
        case_mask = (lower_vars.str.contains('new') 
                     & (lower_vars.str.contains('case') | lower_vars.str.contains('suspect') 
                                                        | lower_vars.str.contains('confirm')) 
                     & ~lower_vars.str.contains('non')
                     & ~lower_vars.str.contains('total'))
        
        keep_data = keep_data[case_mask].dropna()
        
        # Convert data types
        keep_data['date'] = pd.to_datetime(keep_data.date)
        keep_data['totals'] = keep_data.totals.astype(int)
        
        # Assign country label and append to datasets list
        datasets.append(keep_data.assign(country=country))
        

Getting guinea data files
	2014-08-04
	2014-08-26
	2014-08-27
	2014-08-30
	2014-08-31
	2014-09-02
	2014-09-04
	2014-09-07
	2014-09-08
	2014-09-09
	2014-09-11
	2014-09-14
	2014-09-16
	2014-09-17
	2014-09-19
	2014-09-21
	2014-09-22
	2014-09-23
	2014-09-24
	2014-09-26
	2014-09-30
	2014-10-01
Getting sl data files
	2014-11-13
	2014-11-14
	2014-11-15
	2014-11-16
	2014-11-17
	2014-11-28
	2014-11-29
Getting liberia data files
	2014-07-01
	2014-07-02
	2014-07-03
	2014-07-07
	2014-07-08
	2014-07-10
	2014-07-13
	2014-07-17
	2014-07-20
	2014-07-24
	2014-07-26
	2014-08-02
	2014-08-04
	2014-08-12
	2014-08-15
	2014-08-17
	2014-08-18
	2014-08-20
	2014-08-25
	2014-08-28
	2014-09-01
	2014-09-02
	2014-09-03
	2014-09-04
	2014-09-05
	2014-09-06
	2014-09-07
	2014-09-08
	2014-09-10
	2014-09-11
	2014-09-12
	2014-09-13
	2014-09-14
	2014-09-15
	2014-09-16
	2014-11-26
	2014-11-27
	2014-11-28
	2014-11-29
	2014-11-30


Now that we have a list populated with `DataFrame` objects for each day and country, we can call `concat` to concatenate them into a single `DataFrame`.

In [15]:
datasets[0]

,date,variable,totals,country
0,2014-08-04,New cases of suspects,5,guinea
1,2014-08-04,New cases of probables,0,guinea
2,2014-08-04,New cases of confirmed,4,guinea
9,2014-08-04,New deaths registered today (confirmed),2,guinea
11,2014-08-04,New deaths registered today (suspects),0,guinea


In [17]:
all_data = pd.concat(datasets)
all_data.head(10)

,date,variable,totals,country
0,2014-08-04,New cases of suspects,5,guinea
1,2014-08-04,New cases of probables,0,guinea
2,2014-08-04,New cases of confirmed,4,guinea
9,2014-08-04,New deaths registered today (confirmed),2,guinea
11,2014-08-04,New deaths registered today (suspects),0,guinea
0,2014-08-26,New cases of suspects,18,guinea
2,2014-08-26,New cases of confirmed,10,guinea
15,2014-08-26,New cases of confirmed among health workers,0,guinea
0,2014-08-27,New cases of suspects,12,guinea
1,2014-08-27,New cases of probables,0,guinea


This works because the structure of each table was identical

### Manipulating indices

Notice from above, however, that the index contains redundant integer index values. We can confirm this:

In [18]:
all_data.index.is_unique

False

We can create a new unique index by calling the `reset_index` method on the new data frame after we import it, which will generate a new ordered, unique index.

In [19]:
all_data = pd.concat(datasets).reset_index(drop=True)
all_data.head(10)

,date,variable,totals,country
0,2014-08-04,New cases of suspects,5,guinea
1,2014-08-04,New cases of probables,0,guinea
2,2014-08-04,New cases of confirmed,4,guinea
3,2014-08-04,New deaths registered today (confirmed),2,guinea
4,2014-08-04,New deaths registered today (suspects),0,guinea
5,2014-08-26,New cases of suspects,18,guinea
6,2014-08-26,New cases of confirmed,10,guinea
7,2014-08-26,New cases of confirmed among health workers,0,guinea
8,2014-08-27,New cases of suspects,12,guinea
9,2014-08-27,New cases of probables,0,guinea


**Reindexing** allows users to manipulate the data labels in a DataFrame. It forces a DataFrame to conform to the new index, and optionally, fill in missing data if requested.

A simple use of `reindex` is to alter the order of the rows. For example, records are currently ordered first by country then by day, since this is the order in which they were iterated over and imported. We might arbitrarily want to reverse the order, which is performed by passing the appropriate index values to `reindex`.

In [20]:
all_data.reindex(all_data.index[::-1])

,date,variable,totals,country
246,2014-11-30,New case/s (confirmed),10,liberia
245,2014-11-30,Newly Reported Cases in HCW,1,liberia
244,2014-11-29,New case/s (confirmed),10,liberia
243,2014-11-29,New Case/s (Probable),4,liberia
242,2014-11-29,New Case/s (Suspected),7,liberia
241,2014-11-29,Newly Reported Cases in HCW,0,liberia
240,2014-11-28,New case/s (confirmed),7,liberia
239,2014-11-28,Newly Reported Cases in HCW,0,liberia
238,2014-11-27,New case/s (confirmed),12,liberia
237,2014-11-27,New Case/s (Probable),9,liberia


Notice that the reindexing operation is not performed "in-place"; the original `DataFrame` remains as it was, and the method returns a copy of the `DataFrame` with the new index. This is a common trait for pandas, and is a Good Thing.

We may also wish to reorder the columns this way.

In [21]:
all_data.reindex(columns=['date', 'country', 'variable', 'totals']).head()

,date,country,variable,totals
0,2014-08-04,guinea,New cases of suspects,5
1,2014-08-04,guinea,New cases of probables,0
2,2014-08-04,guinea,New cases of confirmed,4
3,2014-08-04,guinea,New deaths registered today (confirmed),2
4,2014-08-04,guinea,New deaths registered today (suspects),0


## Group by operations

One of pandas' most powerful features is the ability to perform operations on subgroups of a `DataFrame`. These so-called **group by** operations defines subunits of the dataset according to the values of one or more variabes in the `DataFrame`.

For this data, we want to sum the new case counts by day and country; so we pass these two column names to the `groupby` method, then sum the `totals` column accross them.

In [23]:
all_data_grouped = all_data.groupby(['country', 'date'])
all_data_grouped.head()

,date,variable,totals,country
0,2014-08-04,New cases of suspects,5,guinea
1,2014-08-04,New cases of probables,0,guinea
2,2014-08-04,New cases of confirmed,4,guinea
3,2014-08-04,New deaths registered today (confirmed),2,guinea
4,2014-08-04,New deaths registered today (suspects),0,guinea
5,2014-08-26,New cases of suspects,18,guinea
6,2014-08-26,New cases of confirmed,10,guinea
7,2014-08-26,New cases of confirmed among health workers,0,guinea
8,2014-08-27,New cases of suspects,12,guinea
9,2014-08-27,New cases of probables,0,guinea


In [ ]:
daily_cases = all_data_grouped['totals'].sum()
daily_cases.head(10)

The resulting series retains a hierarchical index from the group by operation. Hence, we can index out the counts for a given country on a particular day by indexing with the appropriate tuple.

In [ ]:
daily_cases[('liberia', '2014-09-02')]

One issue with the data we have extracted is that there appear to be serious **outliers** in the Liberian counts. The values are much too large to be a daily count, even during a serious outbreak.

In [ ]:
daily_cases.sort_values(ascending=False)
daily_cases.head(10)

We can filter these outliers using an appropriate threshold.

In [ ]:
daily_cases = daily_cases[daily_cases<200]

## Plotting

pandas data structures have high-level methods for creating a variety of plots, which tends to be easier than generating the corresponding plot using matplotlib. 

For example, we may want to create a plot of the cumulative cases for each of the three countries. The easiest way to do this is to remove the hierarchical index, and create a `DataFrame` of three columns, which will result in three lines when plotted.

First, call `unstack` to remove the hierarichical index:

In [ ]:
daily_cases.unstack().head()

Next, transpose the resulting `DataFrame` to swap the rows and columns.

In [ ]:
daily_cases.unstack().T.head()

Since we have missing values for some dates, we will assume that the counts for those days were zero (the actual counts for that day may have bee included in the next reporting day's data).

In [ ]:
daily_cases.unstack().T.fillna(0).head()

Finally, calculate the cumulative sum for all the columns, and generate a line plot, which we get by default.

In [ ]:
daily_cases.unstack().T.fillna(0).cumsum().plot()

## Resampling

An alternative to filling days without case reports with zeros is to aggregate the data at a coarser time scale. New cases are often reported by week; we can use the `resample` method to summarize the data into weekly values.

In [ ]:
weekly_cases = daily_cases.unstack().T.resample('W').sum()
weekly_cases

In [ ]:
weekly_cases.cumsum().plot()